# T3

# policy gradient

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch 
import torch.nn as nn
import numpy as np
import gym
from torch.optim import AdamW

**1. Parametrización de política**

In [4]:
class Policy(nn.Module):
    
    def __init__(self, dim_states, dim_actions, continuous_control):
        super(Policy, self).__init__()
        # MLP, fully connected layers, ReLU activations, linear ouput activation
        # dim_states -> 64 -> 64 -> dim_actions

        self.layers = nn.Sequential(
            nn.Linear(dim_states, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, dim_actions)
        )
        
        if continuous_control:
            # trainable parameter
            self.log_std = nn.Parameter(torch.zeros(1, dim_actions))


    def forward(self, input):

        # tensor format
        if isinstance(input, torch.Tensor):
            input=input
            
        else:
            input = torch.from_numpy(input).unsqueeze(dim=0).float()
            
        value = self.layers(input)
        
        return value


In [5]:
env = gym.make('Pendulum-v1')
dim_states = env.observation_space.shape[0]
continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n
print(dim_states, dim_actions,continuous_control)

3 1 True


In [6]:
env.action_space.sample()

array([1.3430176], dtype=float32)

In [7]:
RN_policy= Policy(dim_states, dim_actions,continuous_control)
RN_policy

Policy(
  (layers): Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [8]:
RN_policy.log_std

Parameter containing:
tensor([[0.]], requires_grad=True)

In [9]:
s_t=env.reset()
s_t

array([-0.620878  ,  0.78390723, -0.45770848], dtype=float32)

In [10]:
action=RN_policy(s_t)
action

tensor([[0.2561]], grad_fn=<AddmmBackward>)

In [11]:
class PolicyGradients:
    
    def __init__(self, dim_states, dim_actions, lr, gamma, 
                 continuous_control=False, reward_to_go=False, use_baseline=False):
        
        self._learning_rate = lr
        self._gamma = gamma
        
        self._dim_states = dim_states
        self._dim_actions = dim_actions

        self._continuous_control = continuous_control
        self._use_reward_to_go = reward_to_go
        self._use_baseline = use_baseline

        self._policy = Policy(self._dim_states, self._dim_actions, self._continuous_control)
        # Adam optimizer
        self._optimizer = AdamW(self._policy.parameters(), lr=self._learning_rate)

        self._select_action = self._select_action_continuous if self._continuous_control else self._select_action_discrete
        self._compute_loss = self._compute_loss_continuous if self._continuous_control else self._compute_loss_discrete


    def select_action(self, observation):
        return self._select_action(observation)
        

    def _select_action_discrete(self, observation):
        # sample from categorical distribution
        RN_policy=self._policy 
        logits=RN_policy(observation)

        # Probabilidad de cada acción
        probs = torch.softmax(logits, dim=-1)

        # Distribución de probabilidad categorica
        dist = torch.distributions.Categorical(probs)

        # Sample de acción
        action = dist.sample().item()
     
        return action


    def _select_action_continuous(self, observation):
        # sample from normal distribution
        # use the log std trainable parameter

        # RN
        RN_policy=self._policy

        # Parametro log std de la RN
        log_std=RN_policy.log_std
        std = torch.exp(log_std)

        # Politica dada la observación (Representa el promedio de la distribución normal que muestrea acciones)
        policy=RN_policy(observation)
        
        # Distribución normal de parametros mean y std, esta se utiliza para muestrear acciones de modo de tal de explorar el espacio de acciones
        dist = torch.distributions.Normal(policy, std)

        # sample de acción
        action = dist.sample()
        
        # Asegurarse de que las acciones están dentro del rango [-1, 1]
        #action = torch.tanh(action)
        
        return action
            

    def update(self, observation_batch, action_batch, advantage_batch):
        # update the policy here
        # you should use self._compute_loss 

        pass
    

    def _compute_loss_discrete(self, observation_batch, action_batch, advantage_batch):
        # use negative logprobs * advantages
        pass


    def _compute_loss_continuous(self, observation_batch, action_batch, advantage_batch):
        # use negative logprobs * advantages
        pass

    
    def estimate_returns(self, rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:
                
            if self._use_reward_to_go:
                # only for part 2
                estimated_return = None
            else:
                estimated_return = None
            
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if self._use_baseline:
            # only for part 2
            average_return_baseline = None
            # Use the baseline:
            #estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


    # It may be useful to discount the rewards using an auxiliary function [optional]
    def _discount_rewards(self, rewards):
        pass


**2. Muestreo de trayectorias**

In [12]:
import gym
import time
import datetime
import csv

import numpy as np

import matplotlib.pyplot as plt

#from policy_gradients import PolicyGradients


def perform_single_rollout(env, agent, episode_nb, render=False):

    # Modify this function to return a tuple of numpy arrays containing (observations, actions, rewards).
    # (np.array(obs), np.array(acs), np.array(rws))
    # np.array(obs) -> shape: (time_steps, nb_obs)
    # np.array(acs) -> shape: (time_steps, nb_acs) if actions are continuous, (time_steps,) if actions are discrete
    # np.array(rws) -> shape: (time_steps,)

    obs_list = []
    action_list = []
    reward_list = []

    ob_t = env.reset()
    
    done = False
    episode_reward = 0
    nb_steps = 0

    while not done:
        
        if render:
            env.render()
            time.sleep(1. / 60)

        action = agent.select_action(ob_t)
        
        ob_t1, reward, done, _ = env.step(action)

        obs_list.append(ob_t1)
        action_list.append(action)
        reward_list.append(reward)

        ob_t = np.squeeze(ob_t1) # <-- may not be needed depending on gym version

        episode_reward += reward
        
        nb_steps += 1

        if done:
            print(f"Largo del episodio {nb_steps}")
            obs_array = np.array(obs_list)
            action_array = np.array(action_list)
            reward_array = np.array(reward_list)

            return obs_array, action_array, reward_array
    #return None

def sample_rollouts(env, agent, training_iter, min_batch_steps):

    sampled_rollouts = []
    total_nb_steps = 0
    episode_nb = 0
    
    while total_nb_steps < min_batch_steps:

        episode_nb += 1
        #render = training_iter%10 == 0 and len(sampled_rollouts) == 0 # Change training_iter%10 to any number you want
        render=False
        # Use perform_single_rollout to get data 
        # Uncomment once perform_single_rollout works.
        # Return sampled_rollouts
       
        sample_rollout = perform_single_rollout(env, agent, episode_nb, render=render)
        total_nb_steps += len(sample_rollout[0])

        sampled_rollouts.append(sample_rollout)
        
    return sampled_rollouts




In [13]:
env = gym.make('Pendulum-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)



In [14]:
# Rollout
x1=perform_single_rollout(env, policy_gradients_agent, 1000, render=False)
print(x1[0].shape)
print(x1[1].shape)
print(x1[2].shape)

Largo del episodio 200
(200, 3)
(200,)
(200,)


**El número de filas de las observaciones es igual con el largo del episodio, por lo que se concluye el correcto funcionamiento de la función.**

In [15]:
# Sample rollouts
x2=sample_rollouts(env, policy_gradients_agent, 1000, 5000)

Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200
Largo del episodio 200


In [16]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(5000, 3)
(5000,)
(5000,)


**El largo del registro de sample rollout es al menos el número de sample mini batch, se concluye que la función funciona.**

In [17]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)


In [18]:
# Rollout
x1=perform_single_rollout(env, policy_gradients_agent, 1000, render=False)
print(x1[0].shape)
print(x1[1].shape)
print(x1[2].shape)

Largo del episodio 18
(18, 4)
(18,)
(18,)


**El número de filas de las observaciones es igual con el largo del episodio, por lo que se concluye el correcto funcionamiento de la función.**

In [19]:
# Sample rollouts
x2=sample_rollouts(env, policy_gradients_agent, 1000, 5000)

Largo del episodio 15
Largo del episodio 19
Largo del episodio 18
Largo del episodio 19
Largo del episodio 19
Largo del episodio 20
Largo del episodio 13
Largo del episodio 18
Largo del episodio 13
Largo del episodio 23
Largo del episodio 15
Largo del episodio 22
Largo del episodio 20
Largo del episodio 17
Largo del episodio 15
Largo del episodio 26
Largo del episodio 13
Largo del episodio 19
Largo del episodio 20
Largo del episodio 28
Largo del episodio 12
Largo del episodio 12
Largo del episodio 18
Largo del episodio 34
Largo del episodio 26
Largo del episodio 17
Largo del episodio 18
Largo del episodio 18
Largo del episodio 18
Largo del episodio 17
Largo del episodio 13
Largo del episodio 49
Largo del episodio 14
Largo del episodio 34
Largo del episodio 21
Largo del episodio 24
Largo del episodio 17
Largo del episodio 57
Largo del episodio 38
Largo del episodio 13
Largo del episodio 18
Largo del episodio 42
Largo del episodio 24
Largo del episodio 25
Largo del episodio 19
Largo del 

In [20]:
sampled_obs = np.concatenate([x2[i][0] for i in range(len(x2))])
sampled_action = np.concatenate([x2[i][1] for i in range(len(x2))])
sampled_reward = np.concatenate([x2[i][2] for i in range(len(x2))])
print(sampled_obs.shape)
print(sampled_action.shape)
print(sampled_reward.shape)

(5020, 4)
(5020,)
(5020,)


**El largo del registro de sample rollout es al menos el número de sample mini batch, se concluye que la función funciona.**

**3. Estimación de retornos**

In [39]:
def estimate_returns( rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:

            # Largo del episodio (largo del reward)
            n_steps = len(rollout_rew[2])
            estimated_return = np.zeros(n_steps)

            if _use_reward_to_go:
            
                estimated_return = None
            else:
                estimated_return = np.zeros(n_steps)

                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                sum_descount=np.sum(vec_gammas*rollout_rew[2])

                for t in range(n_steps):
                    
                    estimated_return[t] = sum_descount
                    
             
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if _use_baseline:
            pass
            #average_return_baseline = np.mean(estimated_returns)
            #estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


In [ ]:
_gamma=0.99
_use_reward_to_go=False
_use_baseline=False

In [58]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 22)
print("")
print("Vector de retorno")
print(estimate_returns(x2))
print("")
retorno=0
for t,reward in enumerate(x2[0][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 1")
print(retorno)
print("")
retorno=0
for t,reward in enumerate(x2[1][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 2")
print(retorno)
print("")

Largo del episodio 17
Largo del episodio 21

Vector de retorno
[15.705681 15.705681 15.705681 15.705681 15.705681 15.705681 15.705681
 15.705681 15.705681 15.705681 15.705681 15.705681 15.705681 15.705681
 15.705681 15.705681 15.705681 19.027214 19.027214 19.027214 19.027214
 19.027214 19.027214 19.027214 19.027214 19.027214 19.027214 19.027214
 19.027214 19.027214 19.027214 19.027214 19.027214 19.027214 19.027214
 19.027214 19.027214 19.027214]

Retorno Ep 1
15.705680661607312

Retorno Ep 2
19.02721317787414



In [61]:
env=gym.make('Pendulum-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 220)

index_episodio_1=x2[0][1].shape[0]

print("")
print("Muestra vector de retorno ep 1")
print(estimate_returns(x2)[index_episodio_1-1])
print("")
print("")
print("Muestra vector de retorno ep 2")
print(estimate_returns(x2)[index_episodio_1+1])
print("")

retorno=0
for t,reward in enumerate(x2[0][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 1")
print(retorno)
print("")

retorno=0
for t,reward in enumerate(x2[1][2]):
    retorno=retorno+(_gamma**t)*reward
print("Retorno Ep 2")
print(retorno)
print("")

Largo del episodio 200
Largo del episodio 200

Muestra vector de retorno ep 1
-542.56146


Muestra vector de retorno ep 2
-679.08484

Retorno Ep 1
-542.561456700748

Retorno Ep 2
-679.0848542854956



**Se observa que que para ambos ambientes y en cada episodio, los retornos coinciden los calculos obtenidos.**

**4. Policy gradients**

**5. Reducción de varianza**

In [67]:
def estimate_returns( rollouts_rew):
        estimated_returns = []
        for rollout_rew in rollouts_rew:

            # Largo del episodio (largo del reward)
            n_steps = len(rollout_rew[2])
            estimated_return = np.zeros(n_steps)

            if _use_reward_to_go:
                
                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                for t in range(n_steps):

                    sum_descount=np.sum(vec_gammas[t:]*rollout_rew[2][t:])
                    estimated_return[t] = sum_descount
    
            else:

                estimated_return = np.zeros(n_steps)

                vec_gammas=np.array([_gamma**j for j in range(n_steps)])

                sum_descount=np.sum(vec_gammas*rollout_rew[2])

                for t in range(n_steps):
                    
                    estimated_return[t] = sum_descount
                     
            estimated_returns = np.concatenate([estimated_returns, estimated_return])

        if _use_baseline:
            
            average_return_baseline = np.mean(estimated_returns)
            estimated_returns -= average_return_baseline

        return np.array(estimated_returns, dtype=np.float32)


In [69]:
env = gym.make('CartPole-v1')

dim_states = env.observation_space.shape[0]

continuous_control = isinstance(env.action_space, gym.spaces.Box)
dim_actions = env.action_space.shape[0] if continuous_control else env.action_space.n

policy_gradients_agent = PolicyGradients(dim_states=dim_states, 
                                             dim_actions=dim_actions, 
                                             lr=0.005,
                                             gamma=0.99,
                                             continuous_control=continuous_control,
                                             reward_to_go=False,
                                             use_baseline=False)

# Sample rollouts (2 episodios): Ejecutar hasta que se generen solo 2 episodios!!
x2=sample_rollouts(env, policy_gradients_agent, 1000, 22)

Largo del episodio 9
Largo del episodio 49


In [71]:
# Caso base
_gamma=0.99
_use_reward_to_go=False
_use_baseline=False
test=estimate_returns(x2)
test

array([ 8.648275,  8.648275,  8.648275,  8.648275,  8.648275,  8.648275,
        8.648275,  8.648275,  8.648275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275, 38.888275, 38.888275,
       38.888275, 38.888275, 38.888275, 38.888275], dtype=float32)

In [72]:
# Reward to go
_gamma=0.99
_use_reward_to_go=True
_use_baseline=False
test=estimate_returns(x2)
test

array([ 8.648275  ,  7.6482754 ,  6.658275  ,  5.6781754 ,  4.707876  ,
        3.7472804 ,  2.7962902 ,  1.85481   ,  0.9227447 , 38.888275  ,
       37.888275  , 36.898277  , 35.918175  , 34.947876  , 33.98728   ,
       33.03629   , 32.09481   , 31.162746  , 30.240002  , 29.326483  ,
       28.422102  , 27.526764  , 26.640379  , 25.762857  , 24.894112  ,
       24.034054  , 23.182596  , 22.339653  , 21.505138  , 20.67897   ,
       19.861063  , 19.051334  , 18.249704  , 17.45609   , 16.670412  ,
       15.8925905 , 15.122547  , 14.360205  , 13.605486  , 12.858314  ,
       12.118613  , 11.38631   , 10.661329  ,  9.943599  ,  9.233046  ,
        8.529598  ,  7.8331847 ,  7.1437354 ,  6.461181  ,  5.785452  ,
        5.1164804 ,  4.454198  ,  3.798539  ,  3.1494362 ,  2.5068247 ,
        1.8706392 ,  1.2408155 ,  0.61729014], dtype=float32)

In [73]:
# Baseline
_gamma=0.99
_use_reward_to_go=False
_use_baseline=True
test=estimate_returns(x2)
test

array([-25.547586, -25.547586, -25.547586, -25.547586, -25.547586,
       -25.547586, -25.547586, -25.547586, -25.547586,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414,   4.692414,   4.692414,
         4.692414,   4.692414,   4.692414], dtype=float32)

In [74]:
# Baseline & reward-to-go
_gamma=0.99
_use_reward_to_go=True
_use_baseline=True
test=estimate_returns(x2)
test

array([ -7.5071583 ,  -8.507158  ,  -9.497158  , -10.477258  ,
       -11.447557  , -12.408154  , -13.359143  , -14.300623  ,
       -15.232689  ,  22.732843  ,  21.732843  ,  20.742844  ,
        19.762743  ,  18.792444  ,  17.831848  ,  16.880857  ,
        15.939378  ,  15.007312  ,  14.084567  ,  13.17105   ,
        12.266668  ,  11.37133   ,  10.484945  ,   9.607424  ,
         8.738678  ,   7.8786197 ,   7.027162  ,   6.184219  ,
         5.349705  ,   4.5235367 ,   3.7056296 ,   2.8959017 ,
         2.0942712 ,   1.3006568 ,   0.51497865,  -0.2628427 ,
        -1.0328859 ,  -1.7952286 ,  -2.5499477 ,  -3.2971199 ,
        -4.0368204 ,  -4.7691236 ,  -5.494104  ,  -6.2118344 ,
        -6.9223876 ,  -7.6258354 ,  -8.322248  ,  -9.011698  ,
        -9.694252  , -10.369982  , -11.038953  , -11.701235  ,
       -12.3568945 , -13.005997  , -13.648608  , -14.284794  ,
       -14.914618  , -15.538143  ], dtype=float32)

**6. Evaluación del algoritmo**